In [ ]:
#Import

In [12]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Conv2D,Add,MaxPooling2D, Dense, BatchNormalization,Input,Flatten, Dropout,GlobalMaxPooling2D,Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [13]:
os.listdir('./../kaggle/')
path_train = '../kaggle/train'
path_valid = '../kaggle/valid'
path_test  = '../kaggle/test'

train = path_train
validation = path_valid
test = path_test

In [14]:
train_datagen = ImageDataGenerator(rescale = 1./255)#initialize train generator 
                                 
valid_datagen = ImageDataGenerator(rescale = 1.0/255.) #initialize validation generator 

test_datagen = ImageDataGenerator(rescale = 1.0/255.) #initialize validation generator 

In [15]:
train_generator = train_datagen.flow_from_directory(train, target_size=(224,224),batch_size=32,class_mode='categorical')

validation_generator = valid_datagen.flow_from_directory(validation, target_size=(224,224),batch_size=32,class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test, target_size=(224,224),batch_size=32,class_mode='categorical')

Found 70626 images belonging to 450 classes.
Found 2250 images belonging to 450 classes.
Found 2250 images belonging to 450 classes.


In [ ]:
#utilisation inception v3 pour lanalyse d'image 

In [16]:
inception = tf.keras.applications.InceptionV3(weights='imagenet',include_top=False,input_shape=(224,224,3))

2022-11-15 17:46:22.016616: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


87910968/87910968 [==============================] - 11s 0us/step


In [17]:
inception.trainable = True
for layer in inception.layers[:197]:
    layer.trainable = False 

In [18]:
for idx, layer in enumerate(inception.layers):
    print(f' {idx}:  {layer.name}: trainable = {layer.trainable}')

 0:  input_1: trainable = False
 1:  conv2d: trainable = False
 2:  batch_normalization: trainable = False
 3:  activation: trainable = False
 4:  conv2d_1: trainable = False
 5:  batch_normalization_1: trainable = False
 6:  activation_1: trainable = False
 7:  conv2d_2: trainable = False
 8:  batch_normalization_2: trainable = False
 9:  activation_2: trainable = False
 10:  max_pooling2d: trainable = False
 11:  conv2d_3: trainable = False
 12:  batch_normalization_3: trainable = False
 13:  activation_3: trainable = False
 14:  conv2d_4: trainable = False
 15:  batch_normalization_4: trainable = False
 16:  activation_4: trainable = False
 17:  max_pooling2d_1: trainable = False
 18:  conv2d_8: trainable = False
 19:  batch_normalization_8: trainable = False
 20:  activation_8: trainable = False
 21:  conv2d_6: trainable = False
 22:  conv2d_9: trainable = False
 23:  batch_normalization_6: trainable = False
 24:  batch_normalization_9: trainable = False
 25:  activation_6: trainab

In [19]:
last_layer = inception.get_layer('mixed7')

print('last layer output shape: ', last_layer.output_shape)

layer_output = last_layer.output

last layer output shape:  (None, 12, 12, 768)


In [28]:
n_categories = len(os.listdir('./../kaggle/train'))# number of categories
print(n_categories)
 

450


In [27]:
# x  = BatchNormalization()(layer_output)
x = Flatten()(layer_output)
x = Dense(1024,activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
x = Dropout(0.4)(x)
x = Dense(n_categories, activation='softmax')(x)

model = Model(inputs=inception.inputs, outputs=x)

In [29]:
model.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

In [30]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
    
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            epochs = 20,
callbacks=[callback])

Epoch 1/20
 546/2208 [======>.......................] - ETA: 26:21:29 - loss: 5.4730 - accuracy: 0.1533

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()

In [ ]:
plt.plot(history.history['accuracy'], label='acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.legend()

In [ ]:
model.save('model.h5')

In [ ]:
model.evaluate(test_generator)